# 0 - MODULES

All modules, libraries, imports and constants used in this file

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1 - DATA SET READING + PLOTTING


Load the synthetic dataset you have previously downloaded. This dataset has two features and a
class label. Use matplotlib’s scatter() function to plot the dataset on a 2D plane and color the
points based on their class label. How do you expect a decision tree to approach data distributed in
this way?

In [ ]:
def

# MAIN FUNCTION

This is the main function of the program that coordinates everything, it does:
<ol>
<li>The read of the file and the plotting of the points</li>
<li></li>
<li></li>
<li></li>
</ol>